In [ ]:
import time

import cv2
import numpy as np

from src.Constants import MAIN_LOOP_DELAY
from src.Image_EKF import Image_EKF
from src.PositionUpdate import PositionUpdate
from src.global_navigation.global_navigation import AStarNavigation
from src.motion_control.LocalNavigator import LocalNavigator
from src.motion_control.MotionControl import MotionControl
from src.motion_control.ThymioController import ThymioController
from src.vision.vision import ComputerVision

vision = ComputerVision()
image_var = vision.get_map()

global_navigation = AStarNavigation(image_var, 100)
global_path = global_navigation.run()

print(global_path)
#Motion control, filtering, local navigation
thymio = ThymioController()
thymio.connect(timeout=5)
local_navigator = LocalNavigator(thymio)
motion_control = MotionControl(thymio, global_path)

global_kidnapping_flag = False

camera, aruco_dict, aruco_params, newcameramtx, dist, mtx =Image_EKF.initialize_camera_and_calibration()
position_update = PositionUpdate(thymio)

thymio.set_speed(40, 40)
try:
    # Define path (list of waypoints)
    while True:
        # print("_______")
        ret, img = camera.read()

        position_update.path_predicition(newcameramtx, dist, aruco_dict, aruco_params, mtx, img)
        cv2.putText(img, "yaw aruco"+ str(np.round(np.degrees((position_update.kf.Z[2])),3)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(img, "yaw"+str(np.round(np.degrees(position_update.kf.theta))), (400, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        PositionUpdate.draw_path(img,position_update.odom_centers,(255,255,0),10)
        PositionUpdate.draw_path(img, position_update.marker_centers,(255,0,0),5)

        cv2.imshow("ArUco Detection", img)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            thymio.stop()
            break

        if not global_kidnapping_flag and position_update.marker_centers:
            warped,warped_point=Image_EKF.warp_and_transform(img,800,600,point=position_update.marker_centers[-1])
            current_pos_x, current_pos_y = global_navigation.pixel_to_grid(warped_point[0], warped_point[1])
            motion_control.setup_position([current_pos_x, current_pos_y, position_update.kf.theta])

            target_position = motion_control.path[motion_control.path_iterator]
            # target_orientation = motion_control.calculate_target_orientation(target_position)
            # motion_control.align_to_target_orientation(target_orientation)            
            # print(f"current_pos_y: {current_pos_y}  current_pos_x: {current_pos_x}")
            motion_control.move_to_target(target_position, [current_pos_x, current_pos_y, position_update.kf.theta])


        if  global_kidnapping_flag and ret :

            if position_update.marker_centers:
                warped,warped_point=Image_EKF.warp_and_transform(img,800,600,point=position_update.marker_centers[-1])
                current_pos_x, current_pos_y = global_navigation.pixel_to_grid(warped_point[0], warped_point[1])
                new_path = global_navigation.new_path(current_pos_x, current_pos_y)
                motion_control.setup_new_path(new_path, position_update.kf.theta)
                global_kidnapping_flag=False
                thymio.set_speed(40, 40)


        # if not position_update.marker_centers:
        #     continue
        if local_navigator.kidnapping_detect():
            thymio.stop()
            print("thymio stop")
            local_navigator.put_back_detecting()
            print("thymio start again")
            global_kidnapping_flag=True
            position_update.kidnapped= True

        if local_navigator.obstacle_detect():
            thymio.stop()
            local_navigator.handle_obstacle()
            """

            current_pos_x, current_pos_y = global_navigation.pixel_to_grid(warped_point[0], warped_point[1])
            new_path = global_navigation.new_path(current_pos_x, current_pos_y)
            motion_control.setup_new_path(new_path, position_update.kf.theta)
            """

            position_update.kidnapped= True
            global_kidnapping_flag=True

        time.sleep(MAIN_LOOP_DELAY)
        if(len(motion_control.path) <= motion_control.path_iterator):
            thymio.stop()
            break

finally:
    thymio.disconnect()
    cv2.destroyAllWindows()